In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy import optimize
from mpl_toolkits import mplot3d
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import pandas as pd
from matplotlib import rc, animation
import statsmodels.api as sm
from scipy import stats
import est_dir
import time
import tqdm

In [2]:
m = 100
n = 16
lambda_2 = 8

In [3]:
noise_list = np.genfromtxt('noise_list_n=%s_m=%s_lambda_max=%s.csv' %
                                (n, m, lambda_2), delimiter=',')
noise_list

array([12.14062262, 38.3920197 , 54.29451494, 66.49692872])

In [4]:
noise_sd = noise_list[0]
noise_sd

12.140622621224393

In [ ]:
max_func_evals_t_list = [30000, 40000, 50000, 60000,70000,80000]
num = 100
domain = (0, 5)
sp_norms = np.zeros((len(max_func_evals_t_list), num))
fp_norms = np.zeros((2, len(max_func_evals_t_list), num))
sp_func_vals_noise = np.zeros((2, len(max_func_evals_t_list), num))
fp_func_vals_noise = np.zeros((2, len(max_func_evals_t_list), num))
sp_func_vals = np.zeros((len(max_func_evals_t_list), num))
fp_func_vals = np.zeros((2, len(max_func_evals_t_list), num))
time_taken = np.zeros((2, len(max_func_evals_t_list), num))
func_evals_step = np.zeros((2, len(max_func_evals_t_list), num)) 
func_evals_dir = np.zeros((2, len(max_func_evals_t_list), num))
no_pos_missed = np.zeros((len(max_func_evals_t_list), num))
index = 0
for max_func_evals_t in max_func_evals_t_list:
    for j in tqdm.tqdm(range(num)):
        seed = j * 50
        np.random.seed(seed)
        f = est_dir.sphere_f_noise
        minimizer = np.random.uniform(*domain, (m, ))
        centre_point = np.random.uniform(*domain, (m, ))
        matrix = est_dir.sphere_func_params(1, lambda_2, m)
        sp_norms[index, j] = np.linalg.norm(minimizer - centre_point)
        sp_func_vals[index, j] = est_dir.sphere_f(centre_point, minimizer, matrix)
        func_args = (minimizer, matrix, 0, noise_sd)
                        
        np.random.seed(seed + 1)
        (upd_point_LS,
         sp_func_vals_noise[0, index, j],
         fp_func_vals_noise[0, index, j],
         full_time_LS,
         func_evals_step[0, index, j],
         func_evals_dir[0, index, j],
         no_its_LS) = est_dir.calc_its_until_sc(centre_point, f, func_args, n, m,
                                                              option='LS',
                                                              max_func_evals=max_func_evals_t)

        fp_norms[0, index, j] = np.linalg.norm(minimizer - upd_point_LS)
        fp_func_vals[0, index, j] =  est_dir.sphere_f(upd_point_LS, minimizer, matrix)                                                 
        np.random.seed(seed + 1)
        (upd_point_XY, 
         sp_func_vals_noise[1, index, j],
         fp_func_vals_noise[1, index, j],
         full_time_XY,
         func_evals_step[1, index, j],
         func_evals_dir[1, index, j],
         its_XY) = est_dir.calc_its_until_sc(centre_point, f, func_args, n, m, 
                                             option='XY',
                                             max_func_evals=max_func_evals_t)
        fp_norms[1, index, j] = np.linalg.norm(minimizer - upd_point_XY)
        fp_func_vals[1, index, j] = est_dir.sphere_f(upd_point_XY, minimizer, matrix)
    index += 1

 46%|█████████████████████████████████████▎                                           | 46/100 [08:37<09:50, 10.94s/it]

In [ ]:
ticks = max_func_evals_t_list

In [ ]:
import matplotlib.ticker as ticker
def set_box_color(bp, color):
    plt.setp(bp['boxes'], color=color)
    plt.setp(bp['whiskers'], color=color)
    plt.setp(bp['caps'], color=color)
    plt.setp(bp['medians'], color=color)

def create_boxplots(arr1, arr2, arr3, label1, label2, label3, m, n,
                    max_func_evals_t, noise_sd,
                    title):
    plt.figure(figsize=(10, 5))
    if title == 'func_vals':
        plt.yscale("log")
        plt.ylim(0.01, np.max(arr1) + 1000)
    bpl = plt.boxplot(arr1.T,
                      positions=np.array(range(len(arr1)))*3.0-0.7)
    bpc = plt.boxplot(arr2.T,
                      positions=np.array(range(len(arr2)))*3.0+0)
    bpr = plt.boxplot(arr3.T,
                      positions=np.array(range(len(arr3)))*3.0+0.7)
    set_box_color(bpl, 'green')
    set_box_color(bpc, 'purple')
    set_box_color(bpr, 'navy')
    plt.plot([], c='green', label=label1)
    plt.plot([], c='purple', label=label2)
    plt.plot([], c='navy', label=label3)
    plt.legend(bbox_to_anchor=(0.99, 1.025), loc='upper left',
               prop={'size': 12})
    plt.xlabel(r'$u$', size=15)
    plt.xticks(np.arange(0, len(ticks) * 3, 3), ticks, size=15)
    plt.yticks(fontsize=15)
    plt.tight_layout()
#     plt.savefig('%s_m_%s_n_%s_func_evals_%s_%s_%s_sd_%s_diff_lambda_max.pdf' %
#                 (title, m, n, max_func_evals_t, choice_t, no_vars_t, noise_sd))

In [ ]:
labels = [[r'$||x_1 - x_{*}||$',
         r'$||x_{K}^{(LS)} - x_{*}||$',
         r'$||x_{K}^{(MY)} - x_{*}||$'],
        [r'$f(x_1)$',
         r'$f(x_{K}^{(LS)})$',
         r'$f(x_{K}^{(MY)})$']]

In [ ]:
create_boxplots(sp_norms, fp_norms[0], fp_norms[1], labels[0][0], labels[0][1], labels[0][2], m, n,
                max_func_evals_t, noise_sd, 'norms')

In [ ]:
np.median(fp_norms[1][0])

In [ ]:
np.median(fp_norms[1][1])

In [ ]:
np.median(fp_norms[1][2])

In [ ]:
np.median(fp_norms[1][3])

In [ ]:
create_boxplots(sp_func_vals, fp_func_vals[0], fp_func_vals[1], labels[1][0], labels[1][1], labels[1][2], m, n,
                max_func_evals_t, noise_sd, 'func_vals')

In [ ]:
np.median(fp_func_vals[1][0])

In [ ]:
np.median(fp_func_vals[1][1])

In [ ]:
np.median(fp_func_vals[1][2])

In [ ]:
np.median(fp_func_vals[1][3])